In [1]:

"""
基于Excel文件的自动点名程序
功能：读取学生名单，随机抽取1/3学生点名，手动确认，输出CSV结果文件
适用于：2025学年Python课程
"""

import pandas as pd
import csv
import os
from datetime import datetime
import time
import random

class ExcelRollCall:
    def __init__(self, excel_file):
        """初始化点名系统"""
        self.excel_file = excel_file
        self.students = []
        self.roll_call_results = []
        self.load_student_data()

    def load_student_data(self):
        """从Excel文件加载学生数据"""
        try:
            # 读取Excel文件
            df = pd.read_excel(self.excel_file)

            # 跳过前3行标题，从第4行开始读取学生数据
            # 第3行是表头：序号、姓名、学号、学院、专业、班级、标签内容
            student_data = df.iloc[3:].copy()

            # 清理数据，移除空行
            student_data = student_data.dropna(subset=[df.columns[2]])  # 姓名列不能为空

            print(f"✅ 成功加载 {len(student_data)} 名学生信息")
            print(f"📁 数据来源: {self.excel_file}")

            # 转换为学生列表
            for _, row in student_data.iterrows():
                student = {
                    '序号': row.iloc[1],  # 序号
                    '姓名': row.iloc[2],  # 姓名
                    '学号': row.iloc[3],  # 学号
                    '学院': row.iloc[4],  # 学院
                    '专业': row.iloc[5],  # 专业
                    '班级': row.iloc[6],  # 班级
                    '标签内容': row.iloc[7] if len(row) > 7 else ''  # 标签内容
                }
                self.students.append(student)

        except Exception as e:
            print(f"❌ 读取Excel文件失败: {e}")
            print("💡 请确保文件路径正确且文件未被其他程序占用")

    def display_student_info(self, student):
        """显示学生信息"""
        print("\n" + "="*60)
        print("📋 学生信息:")
        print(f"   序号: {student['序号']}")
        print(f"   姓名: {student['姓名']}")
        print(f"   学号: {student['学号']}")
        print(f"   学院: {student['学院']}")
        print(f"   专业: {student['专业']}")
        print(f"   班级: {student['班级']}")
        print("="*60)

    def get_attendance_confirmation(self, student):
        """获取出勤确认"""
        while True:
            try:
                choice = input(f"\n❓ {student['姓名']} 同学是否到勤？(Y=到勤/N=缺勤): ").strip().upper()
                if choice in ['Y', 'YES', '到勤', '在', '1']:
                    return '到勤'
                elif choice in ['N', 'NO', '缺勤', '不在', '0']:
                    return '缺勤'
                else:
                    print("❌ 请输入 Y(到勤) 或 N(缺勤)")
            except KeyboardInterrupt:
                print("\n\n👋 点名被用户中断")
                return None

    def run_roll_call(self, class_name):
        """运行点名程序（每次随机抽取1/3学生点名）"""
        if not self.students:
            print("❌ 没有学生数据，无法开始点名")
            return

        total_students = len(self.students)
        num_to_call = max(1, total_students // 3)  # 至少抽1人

        print(f"🎯 开始点名程序（{class_name}，本次随机抽取1/3学生）")
        print(f"📊 总学生数: {total_students}")
        print(f"🎲 本次将随机点名人数: {num_to_call}")
        print("📝 点名规则: 输入 Y 表示到勤，输入 N 表示缺勤")
        print("⏹️  按 Ctrl+C 可以随时停止点名")

        # 随机抽取1/3学生
        selected_students = random.sample(self.students, num_to_call)

        # 开始点名
        for i, student in enumerate(selected_students, 1):
            print(f"\n🔄 正在点名: {i}/{num_to_call}")

            # 显示学生信息
            self.display_student_info(student)

            # 获取出勤确认
            attendance = self.get_attendance_confirmation(student)

            if attendance is None:  # 用户中断
                break

            # 记录点名结果
            result = {
                '序号': student['序号'],
                '姓名': student['姓名'],
                '学号': student['学号'],
                '学院': student['学院'],
                '专业': student['专业'],
                '班级': student['班级'],
                '出勤状态': attendance,
                '点名时间': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            }
            self.roll_call_results.append(result)

            print(f"✅ {student['姓名']}: {attendance}")

        # 显示点名统计
        self.show_statistics()

        # 保存结果到CSV文件
        self.save_to_csv(class_name)

    def show_statistics(self):
        """显示点名统计信息"""
        if not self.roll_call_results:
            return

        total_students = len(self.roll_call_results)
        present_count = sum(1 for r in self.roll_call_results if r['出勤状态'] == '到勤')
        absent_count = total_students - present_count
        attendance_rate = (present_count / total_students * 100) if total_students > 0 else 0

        print("\n" + "="*60)
        print("📊 点名统计报告")
        print("="*60)
        print(f"本次点名人数: {total_students}")
        print(f"到勤人数: {present_count}")
        print(f"缺勤人数: {absent_count}")
        print(f"出勤率: {attendance_rate:.1f}%")
        print("="*60)

        # 显示缺勤学生名单
        absent_students = [r['姓名'] for r in self.roll_call_results if r['出勤状态'] == '缺勤']
        if absent_students:
            print(f"\n❌ 缺勤学生名单: {', '.join(absent_students)}")

        # 显示到勤学生名单
        present_students = [r['姓名'] for r in self.roll_call_results if r['出勤状态'] == '到勤']
        if present_students:
            print(f"\n✅ 到勤学生名单: {', '.join(present_students)}")

    def save_to_csv(self, class_name):
        """保存点名结果到CSV文件"""
        if not self.roll_call_results:
            print("❌ 没有点名数据可保存")
            return

        # 生成文件名（包含日期时间）
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        csv_filename = f"点名结果_{class_name}_{timestamp}.csv"

        try:
            # 写入CSV文件
            with open(csv_filename, 'w', newline='', encoding='utf-8-sig') as csvfile:
                fieldnames = ['序号', '姓名', '学号', '学院', '专业', '班级', '出勤状态', '点名时间']
                writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

                # 写入表头
                writer.writeheader()

                # 写入数据
                for result in self.roll_call_results:
                    writer.writerow(result)

            print(f"\n💾 点名结果已保存到文件: {csv_filename}")
            print(f"📂 文件位置: {os.path.abspath(csv_filename)}")

        except Exception as e:
            print(f"❌ 保存CSV文件失败: {e}")

def main():
    """主函数"""
    # 设置文件路径和班级名称
    excel_file = "2025学年专硕Python名单.xls"
    class_name = "专硕Python班"
        
    print(f"📚 将使用名单: {excel_file}")
    print(f"🏫 班级: {class_name}")

    # 检查文件是否存在
    if not os.path.exists(excel_file):
        print(f"❌ 找不到Excel文件: {excel_file}")
        print("💡 请确保文件路径正确")
        return

    try:
        # 创建点名系统实例
        roll_call = ExcelRollCall(excel_file)

        # 运行点名程序
        roll_call.run_roll_call(class_name)

        print("\n👋 点名程序结束，感谢使用！")

    except Exception as e:
        print(f"❌ 程序运行出错: {e}")
        print("💡 请确保已安装 pandas 库: pip install pandas openpyxl")

if __name__ == "__main__":
    main()


📚 将使用名单: 2025学年专硕Python名单.xls
🏫 班级: 专硕Python班
✅ 成功加载 35 名学生信息
📁 数据来源: 2025学年专硕Python名单.xls
🎯 开始点名程序（专硕Python班，本次随机抽取1/3学生）
📊 总学生数: 35
🎲 本次将随机点名人数: 11
📝 点名规则: 输入 Y 表示到勤，输入 N 表示缺勤
⏹️  按 Ctrl+C 可以随时停止点名

🔄 正在点名: 1/11

📋 学生信息:
   序号: 18
   姓名: 田倍嘉
   学号: 2025121279
   学院: 金融科技学院
   专业: 金融
   班级: nan
✅ 田倍嘉: 到勤

🔄 正在点名: 2/11

📋 学生信息:
   序号: 8
   姓名: 王童菲
   学号: 2025121269
   学院: 金融科技学院
   专业: 金融
   班级: nan
✅ 王童菲: 到勤

🔄 正在点名: 3/11

📋 学生信息:
   序号: 15
   姓名: 孙祥超
   学号: 2025121276
   学院: 金融科技学院
   专业: 金融
   班级: nan
✅ 孙祥超: 到勤

🔄 正在点名: 4/11

📋 学生信息:
   序号: 27
   姓名: 刘晓峰
   学号: 2025121288
   学院: 金融科技学院
   专业: 金融
   班级: nan
✅ 刘晓峰: 到勤

🔄 正在点名: 5/11

📋 学生信息:
   序号: 10
   姓名: 朱钥琪
   学号: 2025121271
   学院: 金融科技学院
   专业: 金融
   班级: nan
✅ 朱钥琪: 到勤

🔄 正在点名: 6/11

📋 学生信息:
   序号: 16
   姓名: 吕娜
   学号: 2025121277
   学院: 金融科技学院
   专业: 金融
   班级: nan
✅ 吕娜: 到勤

🔄 正在点名: 7/11

📋 学生信息:
   序号: 2
   姓名: 李雯
   学号: 2025121263
   学院: 金融科技学院
   专业: 金融
   班级: nan
✅ 李雯: 到勤

🔄 正在点名: 8/11

📋 学生信息:
   序号: 31
   姓名: 单双
   学号: 2025121